In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import evaluate
from utils import save_response

# Llama 3 - 8B Instruct Model

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

c:\Users\colin\OneDrive\Desktop\Applied ML\NLP Assignment\Natural_Language_Assignment\.venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\colin\.cache\huggingface\hub\models--meta-llama--Meta-Llama-3-8B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [3]:
messages = [
    {"role": "system", "content": f"""You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style."""},
    {"role": "user", "content": f"""Can you help me write a story with the following details:
Prompt: A zombie horror story set in New York City.
Distance: 2 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint"""},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [4]:
hyperparams = {
    "max_new_tokens": 512, 
    "temperature":0.6, 
    "top_p":0.9, 
    "do_sample": True
}
outputs = model.generate(
    input_ids,
    max_new_tokens=hyperparams["max_new_tokens"],
    eos_token_id=terminators,
    do_sample=hyperparams["do_sample"],
    temperature=hyperparams["temperature"],
    top_p=hyperparams["top_p"],
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\colin\OneDrive\Desktop\Applied ML\NLP Assignment\Natural_Language_Assignment\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
response = outputs[0][input_ids.shape[-1]:]
output = tokenizer.decode(response, skip_special_tokens=True)
print(output)

Here's a zombie horror story set in New York City for a 0.5-mile training session with a short-distance sprint training style.

**Story: "The Brooklyn Bridge Apocalypse"**

You're a survivor of a zombie outbreak in New York City. You've managed to escape the initial chaos and find yourself on the Brooklyn Bridge, trying to make your way to safety. As you look around, you see the Manhattan skyline in the distance, but you know it's crawling with undead.

You start your journey, your heart racing with every step. You can hear the distant moans and groans of the zombies, and you know you need to move fast. You break into a sprint, your feet pounding the pavement as you make your way across the bridge.

**Cue: Speed up to 5 mph**

As you run, you notice the zombies are getting closer. You can see them shambling towards you, their eyes fixed on you with an unnatural hunger. You pick up speed, your breathing heavy as you try to outrun the undead.

Suddenly, you hear a loud thud behind you. Y

In [1]:
save_response("LLAMA3-8B-INSTRUCT", hyperparams, output)

NameError: name 'save_response' is not defined

In [6]:
# perplexity = evaluate.load("perplexity", module_type="metric")
# input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
# results = perplexity.compute(model_id=model_id,
#                              add_start_token=False,
#                              predictions=input_texts, 
#                              device="cuda")
# print(list(results.keys()))

In [7]:
# print(round(results["mean_perplexity"], 2))